### Class Definition

In [4]:
import torch
import torch.nn as nn

class BurgersEquationNN(nn.Module):
    def __init__(self, layers):
        super(BurgersEquationNN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i + 1]))
    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = torch.tanh(self.layers[i](x))
        x = self.layers[-1](x)
        return x

### Loss Function

In [5]:
def burgers_loss(model, x, y, t, nu):
    # Ensure x, y, and t require gradients
    x.requires_grad = True
    y.requires_grad = True
    t.requires_grad = True

    u_v = model(torch.cat([x, y, t], dim=1))
    u = u_v[:, 0:1]
    v = u_v[:, 1:2]

    # Compute u_t and v_t
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    # Compute u_x, u_y and v_x, v_y
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    # Compute u_xx, u_yy and v_xx, v_yy
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]

    # Burgers' equation residuals
    f_u = u_t + u*u_x + v*u_y - nu*(u_xx + u_yy)
    f_v = v_t + u*v_x + v*v_y - nu*(v_xx + v_yy)

    # Loss (MSE of residuals)
    physics_loss_val = torch.mean(f_u**2) + torch.mean(f_v**2)
    return physics_loss_val

### Training Loop

In [6]:
import matplotlib.pyplot as plt
import numpy as np

model = BurgersEquationNN([3, 20, 20, 20, 2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
nu = 0.01

# Training data
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
t = np.linspace(0, 1, 100)
X, Y, T = np.meshgrid(x, y, t)
X = X.flatten()[:, None]
Y = Y.flatten()[:, None]
T = T.flatten()[:, None]

x_data = torch.tensor(X, dtype=torch.float32, requires_grad=True)
y_data = torch.tensor(Y, dtype=torch.float32, requires_grad=True)
t_data = torch.tensor(T, dtype=torch.float32,  requires_grad=True)

def plot_training_losses(losses):
    plt.figure(figsize=(10, 5))
    plt.plot(losses, label="Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

losses = []
for epoch in range(10):
    optimizer.zero_grad()
    loss = burgers_loss(model, x_data, y_data, t_data, nu)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    if epoch % 1 == 0:
        print(f"Epoch {epoch}, Loss {loss.item()}")
plot_training_losses(losses)

KeyboardInterrupt: 